In [1]:
from pyspark.sql import SparkSession

In [2]:
spark=SparkSession.builder.appName('PracDF7').getOrCreate()

In [4]:
df=spark.read.option('header','true').csv('tips.csv',inferSchema=True)

In [5]:
df.show()

+----------+----+------+------+---+------+----+
|total_bill| tip|   sex|smoker|day|  time|size|
+----------+----+------+------+---+------+----+
|     16.99|1.01|Female|    No|Sun|Dinner|   2|
|     10.34|1.66|  Male|    No|Sun|Dinner|   3|
|     21.01| 3.5|  Male|    No|Sun|Dinner|   3|
|     23.68|3.31|  Male|    No|Sun|Dinner|   2|
|     24.59|3.61|Female|    No|Sun|Dinner|   4|
|     25.29|4.71|  Male|    No|Sun|Dinner|   4|
|      8.77| 2.0|  Male|    No|Sun|Dinner|   2|
|     26.88|3.12|  Male|    No|Sun|Dinner|   4|
|     15.04|1.96|  Male|    No|Sun|Dinner|   2|
|     14.78|3.23|  Male|    No|Sun|Dinner|   2|
|     10.27|1.71|  Male|    No|Sun|Dinner|   2|
|     35.26| 5.0|Female|    No|Sun|Dinner|   4|
|     15.42|1.57|  Male|    No|Sun|Dinner|   2|
|     18.43| 3.0|  Male|    No|Sun|Dinner|   4|
|     14.83|3.02|Female|    No|Sun|Dinner|   2|
|     21.58|3.92|  Male|    No|Sun|Dinner|   2|
|     10.33|1.67|Female|    No|Sun|Dinner|   3|
|     16.29|3.71|  Male|    No|Sun|Dinne

In [6]:
df.printSchema()

root
 |-- total_bill: double (nullable = true)
 |-- tip: double (nullable = true)
 |-- sex: string (nullable = true)
 |-- smoker: string (nullable = true)
 |-- day: string (nullable = true)
 |-- time: string (nullable = true)
 |-- size: integer (nullable = true)



In [7]:
df.columns

['total_bill', 'tip', 'sex', 'smoker', 'day', 'time', 'size']

In [8]:
### handling categorical Features

from pyspark.ml.feature import StringIndexer

In [11]:
indexer=StringIndexer(inputCols=['sex','smoker', 'day', 'time'],
                                 outputCols=['sex_indexed','smoker_indexed',
                                            'day_indexed','time_indexed'])
df_r=indexer.fit(df).transform(df)
df_r.show()

+----------+----+------+------+---+------+----+-----------+--------------+-----------+------------+
|total_bill| tip|   sex|smoker|day|  time|size|sex_indexed|smoker_indexed|day_indexed|time_indexed|
+----------+----+------+------+---+------+----+-----------+--------------+-----------+------------+
|     16.99|1.01|Female|    No|Sun|Dinner|   2|        1.0|           0.0|        1.0|         0.0|
|     10.34|1.66|  Male|    No|Sun|Dinner|   3|        0.0|           0.0|        1.0|         0.0|
|     21.01| 3.5|  Male|    No|Sun|Dinner|   3|        0.0|           0.0|        1.0|         0.0|
|     23.68|3.31|  Male|    No|Sun|Dinner|   2|        0.0|           0.0|        1.0|         0.0|
|     24.59|3.61|Female|    No|Sun|Dinner|   4|        1.0|           0.0|        1.0|         0.0|
|     25.29|4.71|  Male|    No|Sun|Dinner|   4|        0.0|           0.0|        1.0|         0.0|
|      8.77| 2.0|  Male|    No|Sun|Dinner|   2|        0.0|           0.0|        1.0|         0.0|


In [12]:
from pyspark.ml.feature import VectorAssembler
featureassembler=VectorAssembler(inputCols=["tip","size","sex_indexed","day_indexed","time_indexed"],
                                 outputCol="Independent Features")

In [13]:
output=featureassembler.transform(df_r)

In [14]:
output.show()


+----------+----+------+------+---+------+----+-----------+--------------+-----------+------------+--------------------+
|total_bill| tip|   sex|smoker|day|  time|size|sex_indexed|smoker_indexed|day_indexed|time_indexed|Independent Features|
+----------+----+------+------+---+------+----+-----------+--------------+-----------+------------+--------------------+
|     16.99|1.01|Female|    No|Sun|Dinner|   2|        1.0|           0.0|        1.0|         0.0|[1.01,2.0,1.0,1.0...|
|     10.34|1.66|  Male|    No|Sun|Dinner|   3|        0.0|           0.0|        1.0|         0.0|[1.66,3.0,0.0,1.0...|
|     21.01| 3.5|  Male|    No|Sun|Dinner|   3|        0.0|           0.0|        1.0|         0.0|[3.5,3.0,0.0,1.0,...|
|     23.68|3.31|  Male|    No|Sun|Dinner|   2|        0.0|           0.0|        1.0|         0.0|[3.31,2.0,0.0,1.0...|
|     24.59|3.61|Female|    No|Sun|Dinner|   4|        1.0|           0.0|        1.0|         0.0|[3.61,4.0,1.0,1.0...|
|     25.29|4.71|  Male|    No|S

In [15]:
output.select("Independent Features").show()

+--------------------+
|Independent Features|
+--------------------+
|[1.01,2.0,1.0,1.0...|
|[1.66,3.0,0.0,1.0...|
|[3.5,3.0,0.0,1.0,...|
|[3.31,2.0,0.0,1.0...|
|[3.61,4.0,1.0,1.0...|
|[4.71,4.0,0.0,1.0...|
|[2.0,2.0,0.0,1.0,...|
|[3.12,4.0,0.0,1.0...|
|[1.96,2.0,0.0,1.0...|
|[3.23,2.0,0.0,1.0...|
|[1.71,2.0,0.0,1.0...|
|[5.0,4.0,1.0,1.0,...|
|[1.57,2.0,0.0,1.0...|
|[3.0,4.0,0.0,1.0,...|
|[3.02,2.0,1.0,1.0...|
|[3.92,2.0,0.0,1.0...|
|[1.67,3.0,1.0,1.0...|
|[3.71,3.0,0.0,1.0...|
|[3.5,3.0,1.0,1.0,...|
|(5,[0,1],[3.35,3.0])|
+--------------------+
only showing top 20 rows



In [16]:
finalized_data=output.select("Independent Features","total_bill")

In [17]:
finalized_data.show()

+--------------------+----------+
|Independent Features|total_bill|
+--------------------+----------+
|[1.01,2.0,1.0,1.0...|     16.99|
|[1.66,3.0,0.0,1.0...|     10.34|
|[3.5,3.0,0.0,1.0,...|     21.01|
|[3.31,2.0,0.0,1.0...|     23.68|
|[3.61,4.0,1.0,1.0...|     24.59|
|[4.71,4.0,0.0,1.0...|     25.29|
|[2.0,2.0,0.0,1.0,...|      8.77|
|[3.12,4.0,0.0,1.0...|     26.88|
|[1.96,2.0,0.0,1.0...|     15.04|
|[3.23,2.0,0.0,1.0...|     14.78|
|[1.71,2.0,0.0,1.0...|     10.27|
|[5.0,4.0,1.0,1.0,...|     35.26|
|[1.57,2.0,0.0,1.0...|     15.42|
|[3.0,4.0,0.0,1.0,...|     18.43|
|[3.02,2.0,1.0,1.0...|     14.83|
|[3.92,2.0,0.0,1.0...|     21.58|
|[1.67,3.0,1.0,1.0...|     10.33|
|[3.71,3.0,0.0,1.0...|     16.29|
|[3.5,3.0,1.0,1.0,...|     16.97|
|(5,[0,1],[3.35,3.0])|     20.65|
+--------------------+----------+
only showing top 20 rows



In [19]:
from pyspark.ml.regression import LinearRegression
##train test split
train_data,test_data=finalized_data.randomSplit([0.75,0.25])
regressor=LinearRegression(featuresCol='Independent Features', labelCol='total_bill')
regressor=regressor.fit(train_data)

In [20]:
###coefficients
regressor.coefficients

DenseVector([2.8143, 3.2847, -1.0949, -0.1774, -1.0341])

In [21]:
### intercepts
regressor.intercept

3.7797337348587727

In [22]:
### prdiction
pred_results=regressor.evaluate(test_data)

In [23]:
pred_results.predictions.show()

+--------------------+----------+------------------+
|Independent Features|total_bill|        prediction|
+--------------------+----------+------------------+
|(5,[0,1],[1.25,2.0])|     10.07| 13.86698975521305|
|(5,[0,1],[1.47,2.0])|     10.77| 14.48612748864662|
| (5,[0,1],[1.5,2.0])|     11.59|14.570555361387562|
| (5,[0,1],[2.0,2.0])|     12.69|15.977686573736586|
| (5,[0,1],[2.0,2.0])|     13.81|15.977686573736586|
|(5,[0,1],[2.03,2.0])|     24.27|16.062114446477526|
|(5,[0,1],[2.31,3.0])|     18.69|20.134821920133838|
| (5,[0,1],[3.0,5.0])|     28.15| 28.64609098265721|
|(5,[0,1],[3.08,2.0])|     17.92|19.017089992410476|
|(5,[0,1],[3.16,2.0])|     15.81| 19.24223098638632|
|(5,[0,1],[3.27,2.0])|     17.78|19.551799853103105|
|(5,[0,1],[3.39,2.0])|     11.61| 19.88951134406687|
|(5,[0,1],[4.08,2.0])|     17.92|21.831352417108526|
|(5,[0,1],[4.29,2.0])|     25.21|22.422347526295113|
|(5,[0,1],[6.73,4.0])|     48.27| 35.85857583204007|
| (5,[0,1],[9.0,4.0])|     48.33| 42.246951536

In [24]:
## Performance Metrics
pred_results.meanAbsoluteError,pred_results.meanSquaredError

(4.175918806823607, 29.39836041322685)

In [25]:
pred_results.r2

0.708181473079112